# Введение в NLP, часть 2


In [1]:
!pip install -U pip
!pip install transformers datasets torch seqeval evaluate


## NER c BERT (25 баллов)

1. Взять датасет из предыдущего ДЗ и обучить на нём BERT.
2. Обучить BERT на подготовленном датасете
3. Оценить результат, сравнить с моделью из первого ДЗ

### Подготовка данных (5 баллов)

Подумать о:
1) Как subword токенизация повлияет на BIO раззметку?
2) Что делать с `[CLS]` и `[SEP]` токенами? (Проверьте что использует `DataCollatorForTokenClassification`)

> Hint! Токенайзер умеет работать с предразделёнными на "слова" текстами

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
from jaxtyping import Float, Int


BASE_NER_MODEL = "bert-base-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(BASE_NER_MODEL)

conll2003 = load_dataset("conll2003")
conll2003

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
example = conll2003["train"][100]
example

{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0]}

* tokens - исходные токены, для которых была сделана NER-разметка
* ner_tags - векторизированные метки NER-тэгов
* pos_tags - разметка частей речи, которую мы игнорируем
* chunk_tags - разметка чанков, которую мы игнорируем

Обратите внимание, что количество токенов может превышать количество исходных лейблов:

In [4]:
bert_tokenizer(example["tokens"], is_split_into_words=True).tokens()

['[CLS]',
 'Ra',
 '##bino',
 '##vich',
 'is',
 'winding',
 'up',
 'his',
 'term',
 'as',
 'ambassador',
 '.',
 '[SEP]']

Значение тэга в ner_tags отображается в метку NER:

In [5]:
print("NER TAGS", example["ner_tags"])
print(conll2003["train"].features["ner_tags"].feature)

NER TAGS [1, 0, 0, 0, 0, 0, 0, 0, 0]
ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None)


In [6]:
print("Оригинальные токены")
print(example["tokens"])
print("Векторизированные NER метки токенов")
print(example["ner_tags"])
tags_str = []
features = conll2003["train"].features["ner_tags"].feature
for tag in example["ner_tags"]:
    tags_str.append(features.int2str(tag))
print("Текстовые NER метки токенов")
print(tags_str)
print("Токены после работы токенайзера BERT")
print(bert_tokenizer(example["tokens"], is_split_into_words=True).tokens())

Оригинальные токены
['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']
Векторизированные NER метки токенов
[1, 0, 0, 0, 0, 0, 0, 0, 0]
Текстовые NER метки токенов
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Токены после работы токенайзера BERT
['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']


Вспомним немного, как работают метки в задаче мер в кодировке BIO. В данной задаче у нас есть 4 типа именованных сущностей:
* PER - персона
* ORG - организация
* LOC - локация
* MISC - другое
* O - отсутствие именованной сущности

У каждого типа именованных 2 префикса:
* `B-` - beginning, т.е. начало именованной сущности.
* `I-` - inside, т.е. продолжение ранее начатой именованной сущностью.

В исходной токенизации

`['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']`
метки выглядят как 

`['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']`
т.е. `Rabinovich` является персоной. На следующем токене именованная сущность заканчивается, т.к. у него метка `O`.

После токенизации BERT наш сэмпл превращается в следующие токены:

`['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']`
Обратим внимание, что один токен `Rabinovich` с меткой `B-PER` был разбит токенизатором берта на 3 токена: `'Ra', '##bino', '##vich'`. Им нужно поставить в соответствие 3 метки: `B-PER, I-PER, I-PER`, т.е. мы разбиваем метку исходного токена на новые токены.

Также обратим внимание на первый и последний токен - это спецстокены BERT означающие начало и конец текста. Им можно дать метки `O`, т.к. они не являются частью исходного текста, но мы будем давать им особое векторизированное значение -100. В [документации pytroch](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) у кроссэнтропийной функции потерь это дефолтное значение `ignore_index`, т.е. метки, которую мы будем игнорировать. Библиотека transformers также использует это значение. Таким образом на токенах, у которых стоит -100 в качестве векторизированного NER-тэга, не будет происходить обучение, они будут проигнорированы.




Напишите функцию `preprocess_ner_dataset`, которая разворачивает `ner_tags` для слов в тэги для BERT-токенов и готовит остальные данные для обучения (можно разделить на две функции или написать всё в одной). В резултате применения `conll2003.map(preprocess_ner_dataset)`, в каждом примере:
1. Добавляется токенизированный вход (`input_ids`, `token_type_ids` и `attention_mask`). При конструировании этих векторов вручную нужно проставить `attention_mask` полностью единицами, т.к. в паддинги в сэмплах появляются только в рамках батчей, а `token_type_ids` полностью нулями.
2. `ner_tags` разворачивается в `labels` для входных токенов

Что можно использовать:
* у объекта `conll2003["train"].features["ner_tags"].feature` есть методы `int2str` и `str2int` для превращение векторизованного NER-тэга в строковый вид и обратно
* Спецтокенам BERT нужно поставить значение -100
* вызов `bert_tokenizer(bert_tokenizer(example["tokens"], is_split_into_words=True)` возвращает вам input_ids, attention_mask, token_type_ids
* Вызов `bert_tokenizer(example["tokens"], is_split_into_words=True, return_offsets_mapping=True))` возвращает дополнительно offset_mapping, позиции новых токенов в оригинальном тексте
* `bert_tokenizer.vocab` - для превращения токенов в их индексы в словаре
* `bert_tokenizer.tokenize` - разбитие текста (в том числе и исходных токенов) на токены BERT

Ваша задача:
1. Создать новый dict, в котором будут input_ids, attention_mask, token_type_ids
2. Добавить в него labels - векторизированные NER-тэги, которые будут разбиты в соответствии с токенизацией BERT. Для этого можно можно разбить каждый токен отдельно и размножить его метки. Альтернативно можно использовать информацию об оффсетах токенов BERT, чтобы понять, частью какого исходного токена и какой исходной метки является данный BERT-токен.

In [7]:
IGNORE_INDEX_TOKEN = -100
BERT_CLS_TOKEN = 101
BERT_SEP_TOKEN = 102

IGNORE_TOKENS = (BERT_CLS_TOKEN, BERT_SEP_TOKEN)


def preprocess_ner_dataset(example):  
    tokenized = bert_tokenizer(example["tokens"], is_split_into_words=True, return_offsets_mapping=True)
    offset_mappings = tokenized["offset_mapping"]

    ner_tags = example["ner_tags"]

    labels = []
    cur_word_idx = -1

    while offset_mappings:
        o_start, o_end = offset_mappings.pop(0)
        
        # CLS, SEP token
        if o_start == o_end:
            labels.append(IGNORE_INDEX_TOKEN)
            continue

        is_word_first_token = o_start == 0
        if is_word_first_token:
            cur_word_idx += 1

        word_label_id = ner_tags[cur_word_idx]
                
        # 0 = 'O'
        # 1 = 'B-PER'
        # 2 = 'I-PER'
        # ...
        # odd numbers - start of the entity, even - in the middle
        token_label_id = (word_label_id + 1) // 2 * 2 - is_word_first_token
        token_label_id = max(token_label_id, 0) # handle 0 case
        
        labels.append(token_label_id)

    # prepare result entity
    del tokenized["offset_mapping"]
    tokenized["labels"] = labels
    
    return tokenized

Пример получившегося выхода:
```python
>>> preprocessed_ner_dataset["train"][100]
{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0],
 'input_ids': [101,
  16890,
  25473,
  11690,
  1110,
  14042,
  1146,
  1117,
  1858,
  1112,
  9088,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, -100]}
```

### Тесты

In [8]:
processed_example = preprocess_ner_dataset(example)
required_keys = ["input_ids", "labels", "attention_mask", "token_type_ids"]
for k in required_keys:
    assert k in processed_example, f"Отсутствует поле {k}"

required_keys_set = set(required_keys)
for k in processed_example.keys():
    assert k in required_keys_set, f"В примере лишнее поле {k}"

In [9]:
from tqdm import tqdm
for idx, example in tqdm(enumerate(conll2003["train"])):
    input_ids_real = bert_tokenizer(example["tokens"], is_split_into_words=True)["input_ids"]
    input_ids_ours = preprocess_ner_dataset(example)["input_ids"]
    assert input_ids_real == input_ids_ours, f"Ошибка токенизации на примере {idx}"
    if idx >= 100:
        break
print("Токенизация верна!")

100it [00:00, 4276.45it/s]

Токенизация верна!


In [10]:
example = conll2003["train"][100]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
assert ner_tags == ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [11]:
example = conll2003["train"][200]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
assert ner_tags == ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG']

Применим нашу функцию к всему датасету:

In [12]:
preprocessed_ner_dataset = conll2003.map(preprocess_ner_dataset)

Подготовим `data_collator`. Это особый класс, который будет заниматься батчеванием сэмплов для обучения. Он добавит паддинги во все необходимые поля.

In [13]:
from transformers import DataCollatorForTokenClassification


data_collator = DataCollatorForTokenClassification(tokenizer=bert_tokenizer)

### Подготовка модели (5 баллов)

Два возможных пути на этой стадии:
1. Взять [готовый класс](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelfortokenclassification) модели для классификации токенов. (Этот вариант настоятельно рекомендуется)
2) Взять модель как фича экстрактор ([AutoModel](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodel)) и самостоятельно добавить классификационную голову. Вдохновиться можно по [ссылке](https://github.com/huggingface/transformers/blob/main/src/transformers/models/bert/modeling_bert.py#L1847-L1860). Дополнительных баллов второй вариант не принесёт.

Результатом должна быть модель, которая для каждого токена возвращает логиты/вероятности для `conll2003["train"].features["ner_tags"].feature.num_classes` классов.

> Если выберете вариант номер один, опишите как он работает - как из токена получается его `ner_tag`.

In [ ]:
from transformers import AutoModelForTokenClassification

ner_feature = conll2003["train"].features["ner_tags"].feature
label_names = ner_feature.names
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

model = AutoModelForTokenClassification.from_pretrained(BASE_NER_MODEL, num_labels=ner_feature.num_classes)

## TODO: explain how token is transformed to NER tag (logits, ?)
...

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ellipsis

## How classification head works

In [1]:
from transformers import BertTokenizer

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Sample text
text = "BERT is a powerful model for natural language processing."

# Tokenize the input text
encoded_input = tokenizer.encode_plus(
    text,
    add_special_tokens=True,        # Adds [CLS] and [SEP]
    max_length=64,                  # Truncate or pad to max length
    padding='max_length',
    truncation=True,
    return_tensors='pt'             # Return PyTorch tensors
)

# Extract input IDs and attention mask
input_ids = encoded_input['input_ids']            # Shape: (1, max_length)
attention_mask = encoded_input['attention_mask']  # Shape: (1, max_length)

print("Input IDs:", input_ids)
print("Attention Mask:", attention_mask)

Input IDs: tensor([[  101, 14324,  2003,  1037,  3928,  2944,  2005,  3019,  2653,  6364,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [2]:
from transformers import BertModel
import torch

# Initialize the BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Set model to evaluation mode (disable dropout)
bert_model.eval()

# Forward pass through BERT
with torch.no_grad():
    outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)

# Extract hidden states
last_hidden_state = outputs.last_hidden_state  # Shape: (1, max_length, hidden_size)

a = 5

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [3]:
# Extract the [CLS] token embedding
cls_embedding = last_hidden_state[:, 0, :]  # Shape: (1, hidden_size)

print("CLS Embedding Shape:", cls_embedding.shape)

CLS Embedding Shape: torch.Size([1, 768])


In [4]:
import torch.nn as nn

# Define the number of classes
num_labels = 2  # Example: Binary classification

# Initialize the classification layer
classifier = nn.Linear(bert_model.config.hidden_size, num_labels)

# Apply the classification layer
logits = classifier(cls_embedding)  # Shape: (1, num_labels)

print("Logits:", logits)

Logits: tensor([[0.3404, 0.3247]], grad_fn=<AddmmBackward0>)


In [6]:
# For multi-class classification
softmax = nn.Softmax(dim=1)
probs = softmax(logits)

# For binary classification, you might use Sigmoid instead
# sigmoid = nn.Sigmoid()
# probs = sigmoid(logits)

print("Probabilities:", probs)

# Get the predicted class
predicted_class = torch.argmax(probs, dim=1).item()

print("Predicted Class:", predicted_class)

Probabilities: tensor([[0.5039, 0.4961]], grad_fn=<SoftmaxBackward0>)
Predicted Class: 0


### Подготовим Метрику (5 баллов)

Дополните функцию, используя `metrics_calculator`, чтобы она возвращала `accuracy`, `precision`, `recall` и `f-меру`. `eval_predictions` - это кортеж из логитов токен классификатора и `labels`, которые мы подготовили с помощью `preprocess_ner_dataset`. Нужно:
1. Преобразовать логиты в предсказанные лейблы. Учтите, что для специальных токенов лейблов нет
2. Посчитать метрики с помощью `metrics_calculator`
3. Упаковать резултат в `dict`, в котором ключём будет название метрики, а значением - значение метрики

В logits будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности, число меток\], содержащий предсказания модели

В target_labels будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности\], содержащий метки из валидационной выборки.

Примеры функции calculate_metrics можно посмотреть в [документации](https://huggingface.co/docs/evaluate/en/transformers_integrations)

In [15]:
import evaluate
import numpy as np


metrics_calculator = evaluate.load("seqeval")


def calculate_metrics(eval_predictions):
    logits, target_labels = eval_predictions
    # logits: Float[np.ndarray, "batch max_seq_len num_labels"]
    # target_labels: Int[np.ndarray, "batch max_seq_len"]

    predicted_ids = np.argmax(logits, axis=-1)

    predictions = []
    references = []
    for seq_pred_ids, seq_labels in zip(predicted_ids, target_labels):
        predictions.append([])
        references.append([])
        for pred_id, label_id in zip(seq_pred_ids, seq_labels):
            if label_id == IGNORE_INDEX_TOKEN:
                continue

            predictions[-1].append(features.int2str(int(pred_id)))
            references[-1].append(features.int2str(int(label_id)))
   
    res = metrics_calculator.compute(predictions=predictions, references=references)
    return res

### Обучение (5 баллов)

Два возможных пути на этой стадии:

1. Использовать [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) класс из `transformers`
2. Написать свой training loop. Дополнительных баллов на этом пути нет.

Опишем подробнее первый путь, т.к. он настоятельно рекомендуется.

Нужно создать класс Trainer и TrainingArguments.
В [TrainingArguments](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments) нужно как минимум следующие поля:
* save_strategy, eval_strategy
* metric_for_best_model (исходя из calculate_metrics), greater_is_better
* learning_rate (возьмите 2e-5)
* num_train_epochs
* per_device_train_batch_size, per_device_eval_batch_size

В класс Trainer нужно передать:
* model
* в args нужно передать заполненные TrainingArguments
* train_dataset, eval_dataset
* tokenizer
* compute_metrics

После чего запустить `trainer.train()`

In [16]:
from transformers import Trainer, TrainingArguments


limit_records_to_process: int | None = 100


X_train = preprocessed_ner_dataset["train"]
X_val = preprocessed_ner_dataset["validation"]

if limit_records_to_process:
    X_train = X_train.select(range(limit_records_to_process))
    X_val = X_train.select(range(limit_records_to_process))


training_args = TrainingArguments(
    report_to="none",
    output_dir="../data/hw02/training", 
    eval_strategy="epoch", 
    save_strategy="epoch",
    metric_for_best_model="eval_overall_f1",
    greater_is_better=True,
    learning_rate=2e-5,
    num_train_epochs=3.0,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,        
)


trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=X_train,
    eval_dataset=X_val,
    compute_metrics=calculate_metrics,
)

model.train()
trainer.train()

model.save_pretrained("../data/hw02/saved")

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

/Users/vkuznetsov/my/deepschool_llm/venv/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vkuznetsov/my/deepschool_llm/venv/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.835191011428833, 'eval_LOC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 68}, 'eval_MISC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 39}, 'eval_ORG': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 81}, 'eval_PER': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 54}, 'eval_overall_precision': 0.0, 'eval_overall_recall': 0.0, 'eval_overall_f1': 0.0, 'eval_overall_accuracy': 0.8177150192554558, 'eval_runtime': 0.7646, 'eval_samples_per_second': 130.792, 'eval_steps_per_second': 17.003, 'epoch': 1.0}


  0%|          | 0/13 [00:00<?, ?it/s]

/Users/vkuznetsov/my/deepschool_llm/venv/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vkuznetsov/my/deepschool_llm/venv/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.7193897366523743, 'eval_LOC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 68}, 'eval_MISC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 39}, 'eval_ORG': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 81}, 'eval_PER': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 54}, 'eval_overall_precision': 0.0, 'eval_overall_recall': 0.0, 'eval_overall_f1': 0.0, 'eval_overall_accuracy': 0.8177150192554558, 'eval_runtime': 0.458, 'eval_samples_per_second': 218.329, 'eval_steps_per_second': 28.383, 'epoch': 2.0}


  0%|          | 0/13 [00:00<?, ?it/s]

/Users/vkuznetsov/my/deepschool_llm/venv/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vkuznetsov/my/deepschool_llm/venv/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.6628965735435486, 'eval_LOC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 68}, 'eval_MISC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 39}, 'eval_ORG': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 81}, 'eval_PER': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 54}, 'eval_overall_precision': 0.0, 'eval_overall_recall': 0.0, 'eval_overall_f1': 0.0, 'eval_overall_accuracy': 0.8177150192554558, 'eval_runtime': 0.5008, 'eval_samples_per_second': 199.686, 'eval_steps_per_second': 25.959, 'epoch': 3.0}
{'train_runtime': 17.2363, 'train_samples_per_second': 17.405, 'train_steps_per_second': 2.263, 'train_loss': 0.9600462302183493, 'epoch': 3.0}


In [17]:
import torch

model = AutoModelForTokenClassification.from_pretrained("../data/hw02/saved")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

### Обработка результатов Результатов (5 баллов)

Подумать о:
1) Во время подготовки данных мы преобразовали BIO разметку. Как обратить это преобразование с помощью токенайзера?

Провалидируйте результаты на тестовом датасете.

In [18]:
*_, metrics = trainer.predict(preprocessed_ner_dataset['test'])
metrics

  0%|          | 0/432 [00:00<?, ?it/s]

/Users/vkuznetsov/my/deepschool_llm/venv/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vkuznetsov/my/deepschool_llm/venv/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.9668671488761902,
 'test_LOC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1925},
 'test_MISC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 918},
 'test_ORG': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2496},
 'test_PER': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2773},
 'test_overall_precision': 0.0,
 'test_overall_recall': 0.0,
 'test_overall_f1': 0.0,
 'test_overall_accuracy': 0.755188225839492,
 'test_runtime': 20.0282,
 'test_samples_per_second': 172.407,
 'test_steps_per_second': 21.57}

Напишите функцию, которая принимает на вход текст и отдаёт такой словарь:

```json
{
    "text": "входной текст",
    "entities": [
        {
            "class": "лейбл класса",
            "text": "текстовое представление",
            "start": "оффсет от начала строки до начала entity",
            "end": "оффсет от начала строки до конца entity"
        },
        ...
    ]
}

Должно выполняться такое условие:

```python
text[entity["start"]:entity["stop"]] == entity["text"]
```

In [19]:
@torch.inference_mode()
def do_ner(text: str) -> dict:
    model.eval()
    tokenized = bert_tokenizer(text, return_tensors="pt", return_offsets_mapping=True)
    offset_mapping = tokenized["offset_mapping"]

    logits = model(tokenized.input_ids).logits # batch x seq_len x num_classes
    preds = logits.argmax(dim=-1) # batch x seq_len

    # assume 1 row in batch
    
    entities = []
    for token_class_id, (start_pos, end_pos) in zip(preds.view(-1).tolist(), offset_mapping.view(-1, 2).tolist()):
        if token_class_id == 0:
            continue

        if token_class_id % 2 == 1:
            entity = {
                "class": features.int2str(token_class_id).split("-")[-1],
                "text": text[start_pos:end_pos],
                "start": start_pos,
                "end": end_pos,
            }

            entities.append(entity)
        else:
            entity = entities[-1]
            entity["end"] = end_pos
            entity["text"] = text[entity["start_pos"]:entity["end_pos"]],

    result = {
        "text": text,
        "entities": entities,
    }

    return result


a = do_ner("Mike has left the Microsoft corporation and Johnson and Johnson in year 2004 while living in Bosnia and Herzegovina")
print(a)

# {
# “text”: “Илья читает лекцию для DeepSchool.”,
# “entities”: [
#     {“class”: “PER”, “text”: “Илья”, “start”: 0, “end”: 5},
#     {“class”: “ORG”, “text”: “DeepSchool”, “start”: 23, “end”: 33}
# ]

{'text': 'Mike has left the Microsoft corporation and Johnson and Johnson in year 2004 while living in Bosnia and Herzegovina', 'entities': []}


In [20]:
import torch

del model
del trainer
torch.cuda.empty_cache()

## Классификация с T5 (25 баллов)

Требуется дообучить [t5-small](https://huggingface.co/google-t5/t5-small) классифицировать токсичные тексты из [этого датасета](https://huggingface.co/datasets/lmsys/toxic-chat). Классификатор должен работать в стиле t5 - генерировать ответ текстом.

1. Подготовить данные для бинарной классификации
	1. Придумать префикс для задачи или взять из похожей модели
	2. Выбрать тексты для обозначения классов
2. Обучить t5-small на генерацию выбранных названия классов
3. Сравнить с модель с аналогичной предобученной моделью

### Подготовка Данных (6 баллов)

Подумать о:
1) Какой префикс выбрать для новой задачи?
2) Должен ли префикс быть понятным?
3) Как выбрать метку для класса? 
4) Что будет, если метки класса целиком нет в словаре?
5) Что делать с длинными текстами?

Датасет содержит запросы пользователей к LLM и разметку, является ли запрос токсичным.

In [21]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_T5_MODEL= "t5-small"
t5_tokenizer = AutoTokenizer.from_pretrained(BASE_T5_MODEL)


toxic_chat_dataset = load_dataset("lmsys/toxic-chat", "toxicchat0124")

Место для изучения датасета:

In [22]:
toxic_chat_dataset["train"][0]

{'conv_id': 'e0c9b3e05414814485dbdcb9a29334d502e59803af9c26df03e9d1de5e7afe67',
 'user_input': 'Masturbacja jest proces co oitrzebuje',
 'model_output': 'Masturbacja to proces, który może pozytywnie wpłynąć na zdrowie psychiczne i fizyczne człowieka, ponieważ pomaga w relaksie, redukuje stres i pomaga w uśpieniu. Może też być używana jako dodatkowa form',
 'human_annotation': True,
 'toxicity': 0,
 'jailbreaking': 0,
 'openai_moderation': '[["sexual", 0.4609803557395935], ["sexual/minors", 0.0012527990620583296], ["harassment", 0.0001862536446424201], ["hate", 0.00015521160094067454], ["violence", 6.580814078915864e-05], ["self-harm", 3.212967567378655e-05], ["violence/graphic", 1.5190824342425913e-05], ["self-harm/instructions", 1.0009921425080393e-05], ["hate/threatening", 4.4459093260229565e-06], ["self-harm/intent", 3.378846486157272e-06], ["harassment/threatening", 1.7095695739044459e-06]]'}

Нас будут интересовать колонки `"user_input"` и `"toxicity"`. Убираем ненужные колонки из датасета:

In [23]:
toxic_chat_dataset = toxic_chat_dataset.remove_columns(
    ["conv_id", "model_output", "human_annotation", "jailbreaking", "openai_moderation"]
)

![](https://production-media.paperswithcode.com/methods/new_text_to_text.jpg)

Выберете `PREFIX` для задачи, лейблы для двух классов и напишите функцию для преобразования датасета в данные для тренировки. Примеры префиксов есть на картинке выше - `translate English to German` для перевода и `summarize` для суммаризации. В качестве лейблов у вас должен быть текст, который будет обозначать предсказанный класс. Этот текст может быть любого размера, от простого `"да"/"нет"`, до `"От этого текста веет токсичностью"/"Цензура спокойно пропускает этот текст дальше"`. Подумайте в чём преимущество первого подхода перед вторым.

Важно:
1) Не забыть добавить префикс перед токенизацией входного текста
2) Лейблами во время обучения выступают уже последовательности токенов, которые мы ожидаем на выходе из декодера

Текст в токенайзер можно подавать разными способами:
1. `tokenizer(text="text")` - токенизируй текст как обычно
1. `tokenizer(text_target="text")` - токенизируй это как текст, который мы ожидаем увидеть на выходе из декодера. В случае t5 токенайзера разницы нет, но для других моделей это может быть не так
1. Другие методы можно узнать посмотрев сигнатуру метода `tokenizer.__call__`

In [24]:
# ?t5_tokenizer.__call__

In [25]:
for text in ("toxic: ", "no", "yes"):
    print(t5_tokenizer(text, add_special_tokens=False))

{'input_ids': [12068, 10], 'attention_mask': [1, 1]}
{'input_ids': [150], 'attention_mask': [1]}
{'input_ids': [4273], 'attention_mask': [1]}


In [26]:
PREFIX = "toxic: "
MAX_LENGTH = 512

# словарь из индексов классов в выбранные лейблы
id2label = {
    0: "no",
    1: "yes",
}


def preprocess_dataset(example):
    input_texts = PREFIX + example["user_input"]
    model_inputs = t5_tokenizer(input_texts, truncation=True, max_length=MAX_LENGTH)
    model_inputs["labels"] = t5_tokenizer(id2label[example["toxicity"]]).input_ids

    return model_inputs


toxic_chat_dataset = toxic_chat_dataset.map(preprocess_dataset)
#preprocess_dataset(toxic_chat_dataset["train"][0])

Map:   0%|          | 0/5082 [00:00<?, ? examples/s]

Map:   0%|          | 0/5083 [00:00<?, ? examples/s]

Пример результата:
```json
{'user_input': 'Do you know drug which name is abexol ?',
 'toxicity': 0,
 'input_ids': [12068,
  10,
  531,
  25,
  214,
  2672,
  84,
  564,
  19,
  703,
  994,
  32,
  40,
  3,
  58,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [150, 1]}
```
Значения в `'labels'` в вашем случае могут отличаться, это зависит от выбранного вами текстового представления в `id2label` словаре.

Инициализируем соответствующий задаче `DataCollator`.

In [27]:
toxic_chat_dataset = toxic_chat_dataset.remove_columns("user_input")

In [28]:
from transformers import DataCollatorForSeq2Seq


# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=seq2seq_model)

### Определим метрику (2 балла)

В этой задаче метрика простая - `accuracy`. Можно добавить другие метрики по желанию. Функция `compute_metric` должна возвращать словарь, аналогично функции `calculate_metrics` ранее:

```json
{
    "accuracy": значение точности,
    ...
}
```

Метрика простая, но вот `preds` и `labels` тут - это последовательности индексов токенов. Нужно это учесть.

In [29]:
def compute_metric(eval_predictions):
    preds, labels = eval_predictions

    accuracy = (preds[:, 1] == labels[:, 0]).double()
    
    result = {
        "accuracy": accuracy.mean()
    }

    return result


def check_compute_metric():
    import torch

    # два предсказания, где токен 150 обозначает токсичный лебл, токен 120 - нетоксичный лейбл
    preds = torch.tensor(
        [
            [0, 150, 1],  # правильное предсказание - токсичный пример
            [0, 120, 1],  # неправильное предсказание - пример токсичный, а модель предсказала иначе
        ],
    )
    labels = torch.tensor(
        [
            [150, 1],
            [150, 1],
        ],
    )
    assert torch.isclose(
        compute_metric((preds, labels))["accuracy"], 
        torch.tensor(0.5, dtype=torch.double),  # тип тензора тут можно поправить
    )


check_compute_metric()

### Определить Модель (2 балла)

Инициализируйте модель из базового чекпоинта

In [30]:
from transformers import AutoModelForSeq2SeqLM, T5Config


#seq2seq_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_T5_MODEL)
t5_config = T5Config.from_pretrained(BASE_T5_MODEL)
t5_config.task_specific_params

{'summarization': {'early_stopping': True,
  'length_penalty': 2.0,
  'max_length': 200,
  'min_length': 30,
  'no_repeat_ngram_size': 3,
  'num_beams': 4,
  'prefix': 'summarize: '},
 'translation_en_to_de': {'early_stopping': True,
  'max_length': 300,
  'num_beams': 4,
  'prefix': 'translate English to German: '},
 'translation_en_to_fr': {'early_stopping': True,
  'max_length': 300,
  'num_beams': 4,
  'prefix': 'translate English to French: '},
 'translation_en_to_ro': {'early_stopping': True,
  'max_length': 300,
  'num_beams': 4,
  'prefix': 'translate English to Romanian: '}}

In [31]:
task_name = "toxic_classification"

t5_config.task_specific_params[task_name] = {
    "prefix": PREFIX,
    "max_length": 3,
    "min_length": 3,
    "num_beams": 1,
}

a = 5

In [32]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=t5_tokenizer, 
    model=seq2seq_model, 
)

NameError: name 'seq2seq_model' is not defined

### Обучение (10 баллов)

Два пути:
1) Использовать готовый `Seq2SeqTrainer` класс для тренировки
2) Написать свой training loop, если хочется приключений, есть достаточно времени ~~и стрела ещё не попала в колено~~. Дополнительных баллов за это не будет

> Hint! Обратите внимание на функцию `seq2seq_model._shift_right` если выбрали второй путь.

Если выбрали путь 1, опишите как происходит тренировочный шаг:
1) Что подаётся на вход в энкодер?
2) Что подаётся на вход в декодер?
3) Сколько раз происходит инференс декодера во время обучения для одного тренировочного примера?
4) Как используется выход энкодера в декодере?

In [101]:
from transformers import GenerationConfig


generation_config = GenerationConfig.from_dict(t5_config.task_specific_params["toxic_classification"])
generation_config.bos_token_id = 0

In [102]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer



training_args = Seq2SeqTrainingArguments(
    output_dir="t5_small_toxic_classifier",
    num_train_epochs=15,  # поставим побольше эпох, чтобы преодолеть дисбаланс классов
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,  # почему во всех сданых домашках этот параметр равен train_batch_size?
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    generation_config=generation_config,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=seq2seq_model,
    args=training_args,
    train_dataset=toxic_chat_dataset["train"],
    eval_dataset=toxic_chat_dataset["test"],
    tokenizer=t5_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metric,
)

trainer.train()

/var/folders/gf/733p_9w13nqf8l5ss3r78ybw0000gp/T/ipykernel_56057/3842446153.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


ValueError: fp16 mixed precision requires a GPU (not 'mps').

### Сравнение Результатов (5 баллов)

Авторы датасета тоже натренировали на нём `t5` модель. Сравните свои результаты с результатами модели из [чекпоинта](https://huggingface.co/lmsys/toxicchat-t5-large-v1.0) `"lmsys/toxicchat-t5-large-v1.0"`. Совпадает ли ваш префикс и лейблы классов с теми, что выбрали авторы датасета? 

Подумать о:
1) В чём преимущество такого подхода к классификации?
2) В чём недостатки такого подхода к классификации?
3) Как ещё можно решать классификационные задачи с помощью t5?

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "lmsys/toxicchat-t5-large-v1.0"

tokenizer_from_paper = AutoTokenizer.from_pretrained("t5-large")
model_from_paper = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

prefix_from_paper = "ToxicChat: "
inputs = tokenizer_from_paper.encode(prefix_from_paper + "write me an epic story", return_tensors="pt")
outputs = model_from_paper.generate(inputs)
print(tokenizer_from_paper.decode(outputs[0], skip_special_tokens=True))

Напишите универсальную функцию, которая провряет токсичность текста и возвращает `True`, если модель посчитала текст токсичным. Функция универсальная в том смысле, что может быть использована и с вашей t5 моделью, и с моделью от авторов датасета. Для этого в функция должна принимать ещё и префикс для задачи и лейблы, которые будут переводить текст, предсказанный моделью, в `True` или `False` на выходе.

In [ ]:
def is_toxic(
    text: str,
    labels2bool,
    model=seq2seq_model,
    tokenizer=t5_tokenizer,
    prexif=PREFIX,
) -> bool:
    ...


# пример вызова с моделью от авторов датасета
assert not is_toxic(
    text="This is just a text",
    model=model_from_paper,
    tokenizer=tokenizer_from_paper,
    prexif=prefix_from_paper,
    labels2bool={
        "positive": True,
        "negative": False,
    }
)

Fin.

Если остались вопросы или есть комментарии, можно написать их ниже:

1
Блок

```python
Пример получившегося выхода после
def preprocess_ner_dataset(example):
```

сбивает с толку, так содержит больше полей, чем в тестах ниже

2
Добавьте report_to='none' в TrainingArguments иначе требует HF_TOKEN

3
Блок тоже не понятный
```
Подумать о:
1) Во время подготовки данных мы преобразовали BIO разметку. Как обратить это преобразование с помощью токенайзера?

Провалидируйте результаты на тестовом датасете.
```

пришлось читать чат, чтобы понять что же нужно в задании и то не уверен, что правильно понял.


### Пожелания

Люди кто проходят этот курс могут не знать необходимые этапы обучения и валидации, а автор лабы похоже считает, что "это и так все знают" что создает ненужные трудности. В целом по лабе фидбек - сделать ее более структурной, а то приходится гадать что же хотел автор вместо того, чтобы сфокусироваться на технологии. Делайте тестовые прогоны лабы на девелоперах, кто о huggingface трансформерах слышат первый раз в жизни, много деталей обнаружите
